# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [18]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
cities_nhl = cities.copy()
    
# Remove the 'note[]'

cities_nhl['NHL'] = cities_nhl['NHL'].apply(lambda x: re.sub( '\[.*\]', '', x))

# Get the values distinct of '-' and ''

cities_nhl = cities_nhl.loc[~(cities_nhl['NHL'] == '—') & ~(cities_nhl['NHL'] == '')]

# Get the usefull columns and rename for convenience

cities_nhl = cities_nhl[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

# Clean the team field and get only the name of each team

nhl_df['team'] = nhl_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])

nhl_df = nhl_df.loc[nhl_df['year'] == 2018]

nhl_df = nhl_df[nhl_df['team'] != 'Division']

'''

TO CHECK FOR CITIES WITH MORE THAN ONE TEAM OR WRITING DIFFERENCES BETWEEN THE TWO DATAFRAMES AND MAKE THE CORRECTIONS

print(nhl_df['team'].values)

print(cities_nba['NHL'].values)

for i in nhl_df['team']:
    
    if i not in cities_nba['NHL'].values:
        print(i)


'''

cities_nhl["NHL"] = cities_nhl["NHL"].replace({'Rangers Islanders Devils': 'Rangers-Islanders-Devils',
                                           'Kings Ducks':'Kings-Ducks'})

cities_nhl["NHL"] = cities_nhl["NHL"].apply(lambda x: x.split('-'))

cities_nhl = cities_nhl.explode('NHL')

renameDict  = { 'Knights': 'Golden Knights',
                  'Leafs': 'Maple Leafs',
                  'Jackets': 'Blue Jackets',
                  'Wings': 'Red Wings'}

nhl_df['team'] = nhl_df['team'].apply(lambda x: renameDict[x] if x in renameDict else x)


df_nhl = pd.merge( nhl_df, cities_nhl, left_on = 'team', right_on = 'NHL')


df_nhl = df_nhl[['Metropolitan area', 'Population', 'team', 'W', 'L']]

cols =  ['Population', 'W', 'L']

df_nhl[cols] = df_nhl[cols].astype(int)

df_nhl = df_nhl.groupby('Metropolitan area').sum().reset_index()

 
df_nhl.loc[df_nhl['Metropolitan area'] == 'New York City', 'Population'] = 20153634

df_nhl.loc[df_nhl['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

df_nhl['W/L'] = df_nhl['W'] / (df_nhl['W'] + df_nhl['L'])

df_nhl['Population'] = df_nhl['Population'].astype(float)

df_nhl['W/L'] = df_nhl['W/L'].astype(float)

population_by_region = df_nhl['Population']

win_loss_by_region = df_nhl['W/L']

assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
result = stats.pearsonr(population_by_region, win_loss_by_region)[0]

result


0.012308996455744264

In [19]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]



def nhl_correlation(): 

    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    cities_nhl = cities.copy()

    # Remove the 'note[]'

    cities_nhl['NHL'] = cities_nhl['NHL'].apply(lambda x: re.sub( '\[.*\]', '', x))

    # Get the values distinct of '-' and ''

    cities_nhl = cities_nhl.loc[~(cities_nhl['NHL'] == '—') & ~(cities_nhl['NHL'] == '')]

    # Get the usefull columns and rename for convenience

    cities_nhl = cities_nhl[['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

    # Clean the team field and get only the name of each team

    nhl_df['team'] = nhl_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])

    nhl_df = nhl_df.loc[nhl_df['year'] == 2018]

    nhl_df = nhl_df[nhl_df['team'] != 'Division']

    cities_nhl["NHL"] = cities_nhl["NHL"].replace({'Rangers Islanders Devils': 'Rangers-Islanders-Devils',
                                               'Kings Ducks':'Kings-Ducks'})

    cities_nhl["NHL"] = cities_nhl["NHL"].apply(lambda x: x.split('-'))

    cities_nhl = cities_nhl.explode('NHL')

    renameDict  = { 'Knights': 'Golden Knights',
                      'Leafs': 'Maple Leafs',
                      'Jackets': 'Blue Jackets',
                      'Wings': 'Red Wings'}

    nhl_df['team'] = nhl_df['team'].apply(lambda x: renameDict[x] if x in renameDict else x)


    df_nhl = pd.merge( nhl_df, cities_nhl, left_on = 'team', right_on = 'NHL')


    df_nhl = df_nhl[['Metropolitan area', 'Population', 'team', 'W', 'L']]

    cols =  ['Population', 'W', 'L']

    df_nhl[cols] = df_nhl[cols].astype(int)

    df_nhl = df_nhl.groupby('Metropolitan area').sum().reset_index()


    df_nhl.loc[df_nhl['Metropolitan area'] == 'New York City', 'Population'] = 20153634

    df_nhl.loc[df_nhl['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

    df_nhl['W/L'] = df_nhl['W'] / (df_nhl['W'] + df_nhl['L'])
    
    

    population_by_region = df_nhl['Population']

    win_loss_by_region = df_nhl['W/L']

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    result = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    return result

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [20]:
cities_nba = cities.copy()

nba_df=pd.read_csv("assets/nba.csv")


# Remove the 'note[]'

cities_nba['NBA'] = cities_nba['NBA'].apply(lambda x: re.sub( '\[.*\]', '', x))

# Get the values distinct of '-' and ''

cities_nba = cities_nba.loc[~(cities_nba['NBA'] == '—') & ~(cities_nba['NBA'] == '')]

# Get the usefull columns and rename for convenience

cities_nba = cities_nba[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

# Clean the team field and get only the name of each team

nba_df['team'] = nba_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])

nba_df['team'] = nba_df['team'].apply(lambda x: re.sub( '\(.*\)', '', x.strip()))

nba_df = nba_df.loc[nba_df['year'] == 2018]

nba_df = nba_df[nba_df['team'] != 'Division']


'''

TO CHECK FOR CITIES WITH MORE THAN ONE TEAM OR WRITING DIFFERENCES BETWEEN THE TWO DATAFRAMES AND MAKE THE CORRECTIONS

print(nba_df['team'].values)

print(cities_nba['NBA'].values)

for i in nba_df['team']:
    
    if i not in cities_nba['NBA'].values:
        print(i)


'''
cities_nba['NBA'] = cities_nba['NBA'].replace({'Knicks Nets': 'Knicks-Nets',
                                           'Lakers Clippers':'Lakers-Clippers',
                                            'Trail Blazers': 'Blazers'})

cities_nba['NBA'] = cities_nba['NBA'].apply(lambda x: x.split('-'))

cities_nba = cities_nba.explode('NBA')

nba_df['team'] = nba_df['team'].apply(lambda x: re.sub( '\\.*', '', x.strip()))

# New York City 20153634 Los Angeles 13310447


df_nba = pd.merge( nba_df, cities_nba, left_on = 'team', right_on = 'NBA')

df_nba = df_nba[['Metropolitan area', 'Population', 'team', 'W', 'L']]

cols =  ['Population', 'W', 'L']

df_nba[cols] = df_nba[cols].astype(int)

df_nba = df_nba.groupby('Metropolitan area').sum().reset_index()


df_nba.loc[df_nba['Metropolitan area'] == 'New York City', 'Population'] = 20153634

df_nba.loc[df_nba['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

df_nba['W/L'] = df_nba['W'] / (df_nba['W'] + df_nba['L'])
    
    

population_by_region = df_nba['Population']

win_loss_by_region = df_nba['W/L']

df_nba

,Metropolitan area,Population,team,W,L,W/L
0,Atlanta,5789700,Hawks,24,58,0.292683
1,Boston,4794447,Celtics,55,27,0.670732
2,Charlotte,2474314,Hornets,36,46,0.439024
3,Chicago,9512999,Bulls,27,55,0.329268
4,Cleveland,2055612,Cavaliers,50,32,0.609756
5,Dallas–Fort Worth,7233323,Mavericks,24,58,0.292683
6,Denver,2853077,Nuggets,46,36,0.560976
7,Detroit,4297617,Pistons,39,43,0.475610
8,Houston,6772470,Rockets,65,17,0.792683
9,Indianapolis,2004230,Pacers,48,34,0.585366


In [21]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    
    cities_nba = cities.copy()

    nba_df=pd.read_csv("assets/nba.csv")


    # Remove the 'note[]'

    cities_nba['NBA'] = cities_nba['NBA'].apply(lambda x: re.sub( '\[.*\]', '', x))

    # Get the values distinct of '-' and ''

    cities_nba = cities_nba.loc[~(cities_nba['NBA'] == '—') & ~(cities_nba['NBA'] == '')]

    # Get the usefull columns and rename for convenience

    cities_nba = cities_nba[['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

    # Clean the team field and get only the name of each team

    nba_df['team'] = nba_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])

    nba_df['team'] = nba_df['team'].apply(lambda x: re.sub( '\(.*\)', '', x.strip()))

    nba_df = nba_df.loc[nba_df['year'] == 2018]

    nba_df = nba_df[nba_df['team'] != 'Division']

    cities_nba['NBA'] = cities_nba['NBA'].replace({'Knicks Nets': 'Knicks-Nets',
                                               'Lakers Clippers':'Lakers-Clippers',
                                                'Trail Blazers': 'Blazers'})

    cities_nba['NBA'] = cities_nba['NBA'].apply(lambda x: x.split('-'))

    cities_nba = cities_nba.explode('NBA')

    nba_df['team'] = nba_df['team'].apply(lambda x: re.sub( '\\.*', '', x.strip()))


    df_nba = pd.merge( nba_df, cities_nba, left_on = 'team', right_on = 'NBA')

    df_nba = df_nba[['Metropolitan area', 'Population', 'team', 'W', 'L']]

    cols =  ['Population', 'W', 'L']

    df_nba[cols] = df_nba[cols].astype(int)

    df_nba = df_nba.groupby('Metropolitan area').sum().reset_index()


    df_nba.loc[df_nba['Metropolitan area'] == 'New York City', 'Population'] = 20153634

    df_nba.loc[df_nba['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

    df_nba['W/L'] = df_nba['W'] / (df_nba['W'] + df_nba['L'])



    population_by_region = df_nba['Population']

    win_loss_by_region = df_nba['W/L']


    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [22]:
cities_mlb = cities.copy()

mlb_df=pd.read_csv("assets/mlb.csv")

# Remove the 'note[]'

cities_mlb['MLB'] = cities_mlb['MLB'].apply(lambda x: re.sub( '\[.*\]', '', x.strip()))

# Get the values distinct of '-' and ''

cities_mlb = cities_mlb.loc[~(cities_mlb['MLB'] == '—') & ~(cities_mlb['MLB'] == '')]

# Get the usefull columns and rename for convenience

cities_mlb = cities_mlb[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

# Clean the team field and get only the name of each team

mlb_df.loc[mlb_df['team'] == 'Boston Red Sox', 'team'] = 'RSox'


mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])


mlb_df = mlb_df.loc[mlb_df['year'] == 2018]

mlb_df = mlb_df[mlb_df['team'] != 'Division']

mlb_df.loc[mlb_df['team'] == 'RSox', 'team'] = 'Red Sox'

mlb_df.loc[mlb_df['team'] == 'Sox', 'team'] = 'White Sox'


cities_mlb['MLB'] = cities_mlb['MLB'].replace({'Cubs White Sox': 'Cubs-White Sox',
                                           'Yankees Mets': 'Yankees-Mets',
                                            'Dodgers Angels': 'Dodgers-Angels',
                                            'Giants Athletics': 'Giants-Athletics',
                                            'Blue Jays': 'Jays'
                                              })

cities_mlb['MLB'] = cities_mlb['MLB'].apply(lambda x: x.split('-'))

cities_mlb = cities_mlb.explode('MLB')

# REPEATED CITIES: New York City 20153634 Los Angeles 13310447 San Francisco Bay Area 6657982 Chicago 9512999


df_mlb = pd.merge( mlb_df, cities_mlb, left_on = 'team', right_on = 'MLB')

df_mlb = df_mlb[['Metropolitan area', 'Population', 'team', 'W', 'L']]

cols =  ['Population', 'W', 'L']

df_mlb[cols] = df_mlb[cols].astype(int)

df_mlb = df_mlb.groupby('Metropolitan area').sum().reset_index()


df_mlb.loc[df_mlb['Metropolitan area'] == 'New York City', 'Population'] = 20153634

df_mlb.loc[df_mlb['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

df_mlb.loc[df_mlb['Metropolitan area'] == 'San Francisco Bay Area', 'Population'] = 6657982

df_mlb.loc[df_mlb['Metropolitan area'] == 'Chicago', 'Population'] = 9512999

df_mlb['W/L'] = df_mlb['W'] / (df_mlb['W'] + df_mlb['L'])
    

population_by_region = df_mlb['Population']

win_loss_by_region = df_mlb['W/L']

df_mlb

,Metropolitan area,Population,team,W,L,W/L
0,Atlanta,5789700,Braves,90,72,0.555556
1,Baltimore,2798886,Orioles,47,115,0.290123
2,Boston,4794447,Red Sox,108,54,0.666667
3,Chicago,9512999,White SoxCubs,157,168,0.483077
4,Cincinnati,2165139,Reds,67,95,0.413580
5,Cleveland,2055612,Indians,91,71,0.561728
6,Dallas–Fort Worth,7233323,Rangers,67,95,0.413580
7,Denver,2853077,Rockies,91,72,0.558282
8,Detroit,4297617,Tigers,64,98,0.395062
9,Houston,6772470,Astros,103,59,0.635802


In [23]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    
    cities_mlb = cities.copy()

    mlb_df=pd.read_csv("assets/mlb.csv")

    # Remove the 'note[]'

    cities_mlb['MLB'] = cities_mlb['MLB'].apply(lambda x: re.sub( '\[.*\]', '', x.strip()))

    # Get the values distinct of '-' and ''

    cities_mlb = cities_mlb.loc[~(cities_mlb['MLB'] == '—') & ~(cities_mlb['MLB'] == '')]

    # Get the usefull columns and rename for convenience

    cities_mlb = cities_mlb[['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

    # Clean the team field and get only the name of each team

    mlb_df.loc[mlb_df['team'] == 'Boston Red Sox', 'team'] = 'RSox'


    mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])


    mlb_df = mlb_df.loc[mlb_df['year'] == 2018]

    mlb_df = mlb_df[mlb_df['team'] != 'Division']

    mlb_df.loc[mlb_df['team'] == 'RSox', 'team'] = 'Red Sox'

    mlb_df.loc[mlb_df['team'] == 'Sox', 'team'] = 'White Sox'


    cities_mlb['MLB'] = cities_mlb['MLB'].replace({'Cubs White Sox': 'Cubs-White Sox',
                                               'Yankees Mets': 'Yankees-Mets',
                                                'Dodgers Angels': 'Dodgers-Angels',
                                                'Giants Athletics': 'Giants-Athletics',
                                                'Blue Jays': 'Jays'
                                                  })

    cities_mlb['MLB'] = cities_mlb['MLB'].apply(lambda x: x.split('-'))

    cities_mlb = cities_mlb.explode('MLB')

    # New York City 20153634 Los Angeles 13310447 San Francisco Bay Area 6657982 Chicago 9512999


    df_mlb = pd.merge( mlb_df, cities_mlb, left_on = 'team', right_on = 'MLB')

    df_mlb = df_mlb[['Metropolitan area', 'Population', 'team', 'W', 'L']]

    cols =  ['Population', 'W', 'L']

    df_mlb[cols] = df_mlb[cols].astype(int)

    df_mlb = df_mlb.groupby('Metropolitan area').sum().reset_index()


    df_mlb.loc[df_mlb['Metropolitan area'] == 'New York City', 'Population'] = 20153634

    df_mlb.loc[df_mlb['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

    df_mlb.loc[df_mlb['Metropolitan area'] == 'San Francisco Bay Area', 'Population'] = 6657982

    df_mlb.loc[df_mlb['Metropolitan area'] == 'Chicago', 'Population'] = 9512999

    df_mlb['W/L'] = df_mlb['W'] / (df_mlb['W'] + df_mlb['L'])


    population_by_region = df_mlb['Population']

    win_loss_by_region = df_mlb['W/L']
    

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [24]:
cities_nfl = cities.copy()

mlb_df=pd.read_csv("assets/nfl.csv")

# Remove the 'note[]'

cities_nfl['NFL'] = cities_nfl['NFL'].apply(lambda x: re.sub( '\[.*\]', '', x))

# Get the values distinct of '-', '' and '- '

cities_nfl = cities_nfl.loc[~(cities_nfl['NFL'] == '—') & ~(cities_nfl['NFL'] == '') & ~(cities_nfl['NFL'] == '— ')]

# Get the usefull columns and rename for convenience

cities_nfl = cities_nfl[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

# Clean the team field and get only the name of each team

mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip().replace('+', ''))

mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])

mlb_df['team'] = mlb_df['team'].apply(lambda x: re.sub( '\(.*\)', '', x.strip()))

mlb_df = mlb_df.loc[mlb_df['year'] == 2018]

mlb_df = mlb_df[(mlb_df['team'] != 'Division') & (mlb_df['team'] != 'West') & (mlb_df['team'] != 'South') & (mlb_df['team'] != 'East') & (mlb_df['team'] != 'North')]

cities_nfl['NFL'] = cities_nfl['NFL'].replace({'Giants Jets': 'Giants-Jets',
                                           'Rams Chargers':'Rams-Chargers',
                                            '49ers Raiders': '49ers-Raiders'})

cities_nfl['NFL'] = cities_nfl['NFL'].apply(lambda x: x.split('-'))

cities_nfl = cities_nfl.explode('NFL')


df_nfl = pd.merge( mlb_df, cities_nfl, left_on = 'team', right_on = 'NFL')

df_nfl = df_nfl[['Metropolitan area', 'Population', 'team', 'W', 'L']]


cols =  ['Population', 'W', 'L']

df_nfl[cols] = df_nfl[cols].astype(int)

df_nfl = df_nfl.groupby('Metropolitan area').sum().reset_index()


df_nfl.loc[df_nfl['Metropolitan area'] == 'New York City', 'Population'] = 20153634

df_nfl.loc[df_nfl['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

df_nfl.loc[df_nfl['Metropolitan area'] == 'San Francisco Bay Area', 'Population'] = 6657982


df_nfl['W/L'] = df_nfl['W'] / (df_nfl['W'] + df_nfl['L'])
    

population_by_region = df_nfl['Population']

win_loss_by_region = df_nfl['W/L']

df_nfl

,Metropolitan area,Population,team,W,L,W/L
0,Atlanta,5789700,Falcons,7,9,0.437500
1,Baltimore,2798886,Ravens,10,6,0.625000
2,Boston,4794447,Patriots,11,5,0.687500
3,Buffalo,1132804,Bills,6,10,0.375000
4,Charlotte,2474314,Panthers,7,9,0.437500
5,Chicago,9512999,Bears,12,4,0.750000
6,Cincinnati,2165139,Bengals,6,10,0.375000
7,Cleveland,2055612,Browns,7,8,0.466667
8,Dallas–Fort Worth,7233323,Cowboys,10,6,0.625000
9,Denver,2853077,Broncos,6,10,0.375000


In [25]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    
    cities_nfl = cities.copy()

    mlb_df=pd.read_csv("assets/nfl.csv")


    # Remove the 'note[]'

    cities_nfl['NFL'] = cities_nfl['NFL'].apply(lambda x: re.sub( '\[.*\]', '', x))

    # Get the values distinct of '-', '' and '- '

    cities_nfl = cities_nfl.loc[~(cities_nfl['NFL'] == '—') & ~(cities_nfl['NFL'] == '') & ~(cities_nfl['NFL'] == '— ')]

    # Get the usefull columns and rename for convenience

    cities_nfl = cities_nfl[['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']].rename(columns = {'Population (2016 est.)[8]': 'Population'})

    # Clean the team field and get only the name of each team

    mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip().replace('+', ''))

    mlb_df['team'] = mlb_df['team'].apply(lambda x: x.strip().replace('*', '').split(' ')[-1])

    mlb_df['team'] = mlb_df['team'].apply(lambda x: re.sub( '\(.*\)', '', x.strip()))

    mlb_df = mlb_df.loc[mlb_df['year'] == 2018]

    mlb_df = mlb_df[(mlb_df['team'] != 'Division') & (mlb_df['team'] != 'West') & (mlb_df['team'] != 'South') & (mlb_df['team'] != 'East') & (mlb_df['team'] != 'North')]

    cities_nfl['NFL'] = cities_nfl['NFL'].replace({'Giants Jets': 'Giants-Jets',
                                               'Rams Chargers':'Rams-Chargers',
                                                '49ers Raiders': '49ers-Raiders'})

    cities_nfl['NFL'] = cities_nfl['NFL'].apply(lambda x: x.split('-'))

    cities_nfl = cities_nfl.explode('NFL')


    df_nfl = pd.merge( mlb_df, cities_nfl, left_on = 'team', right_on = 'NFL')

    df_nfl = df_nfl[['Metropolitan area', 'Population', 'team', 'W', 'L']]


    cols =  ['Population', 'W', 'L']

    df_nfl[cols] = df_nfl[cols].astype(int)

    df_nfl = df_nfl.groupby('Metropolitan area').sum().reset_index()


    df_nfl.loc[df_nfl['Metropolitan area'] == 'New York City', 'Population'] = 20153634

    df_nfl.loc[df_nfl['Metropolitan area'] == 'Los Angeles', 'Population'] = 13310447

    df_nfl.loc[df_nfl['Metropolitan area'] == 'San Francisco Bay Area', 'Population'] = 6657982


    df_nfl['W/L'] = df_nfl['W'] / (df_nfl['W'] + df_nfl['L'])


    population_by_region = df_nfl['Population']

    win_loss_by_region = df_nfl['W/L']

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]